## Building RNNs is Fun with PyTorch and Google Colab

https://medium.com/dair-ai/building-rnns-is-fun-with-pytorch-and-google-colab-3903ea9a3a79

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
import numpy as np

In [2]:
class SingleRNN(nn.Module):
    def __init__(self, n_inputs, n_neurons):
        super(SingleRNN, self).__init__()
        
        self.Wx = torch.randn(n_inputs, n_neurons) # 4 X 1
        self.Wy = torch.randn(n_neurons, n_neurons) # 1 X 1
        
        self.b = torch.zeros(1, n_neurons) # 1 X 4
    
    def forward(self, X0, X1):
        self.Y0 = torch.tanh(torch.mm(X0, self.Wx) + self.b) # 4 X 1
        
        self.Y1 = torch.tanh(torch.mm(self.Y0, self.Wy) + torch.mm(X1, self.Wx) + self.b) # 4 X 1
        
        return self.Y0, self.Y1

In [3]:
N_INPUT = 4
N_NEURONS = 1

X0_batch = torch.tensor([[0,1,2,0], [3,4,5,0], 
                         [6,7,8,0], [9,0,1,0]],
                        dtype = torch.float) #t=0 => 4 X 4

X1_batch = torch.tensor([[9,8,7,0], [0,0,0,0], 
                         [6,5,4,0], [3,2,1,0]],
                        dtype = torch.float) #t=1 => 4 X 4

In [4]:
X0_batch.shape

torch.Size([4, 4])

In [5]:
model = SingleRNN(N_INPUT, N_NEURONS)

In [7]:
Y0_val, Y1_val = model(X0_batch, X1_batch)

In [9]:
Y0_val

tensor([[0.2961],
        [0.1623],
        [0.0224],
        [0.9993]])

In [10]:
Y1_val

tensor([[-0.8415],
        [-0.2659],
        [-0.5550],
        [-0.9718]])

Now i change the neurons and input

In [17]:
class BasicRNN(nn.Module):
    def __init__(self, n_inputs, n_neurons):
        super(BasicRNN, self).__init__()
        
        self.Wx = torch.randn(n_inputs, n_neurons) # n_inputs X n_neurons
        self.Wy = torch.randn(n_neurons, n_neurons) # n_neurons X n_neurons
        
        self.b = torch.zeros(1, n_neurons) # 1 X n_neurons
    
    def forward(self, X0, X1):
        self.Y0 = torch.tanh(torch.mm(X0, self.Wx) + self.b) # batch_size X n_neurons
        
        self.Y1 = torch.tanh(torch.mm(self.Y0, self.Wy) +
                            torch.mm(X1, self.Wx) + self.b) # batch_size X n_neurons
        
        return self.Y0, self.Y1

In [18]:
N_INPUT = 3 # number of features in input
N_NEURONS = 5 # number of units in layer

X0_batch = torch.tensor([[0,1,2], [3,4,5], 
                         [6,7,8], [9,0,1]],
                        dtype = torch.float) #t=0 => 4 X 3

X1_batch = torch.tensor([[9,8,7], [0,0,0], 
                         [6,5,4], [3,2,1]],
                        dtype = torch.float) #t=1 => 4 X 3

model = BasicRNN(N_INPUT, N_NEURONS)

Y0_val, Y1_val = model(X0_batch, X1_batch)

In [19]:
print(Y0_val)
print(Y1_val)

tensor([[-0.9506,  0.9967,  0.1502, -1.0000, -0.9716],
        [-0.9935,  1.0000, -0.9987, -1.0000, -0.9980],
        [-0.9992,  1.0000, -1.0000, -1.0000, -0.9999],
        [ 1.0000, -0.9766, -1.0000,  1.0000, -0.9995]])
tensor([[-0.9864,  1.0000, -1.0000, -1.0000,  0.9615],
        [-0.9279, -0.2538, -0.8387,  0.8645,  1.0000],
        [-0.9527,  1.0000, -1.0000, -1.0000,  0.9999],
        [-0.9327,  0.5439, -0.4183, -0.9964,  0.9845]])


## Pytorch RNN CELL


In [20]:
rnn = nn.RNNCell(3, 5) # n_input X n_neurons

X_batch = torch.tensor([[[0,1,2], [3,4,5], 
                         [6,7,8], [9,0,1]],
                        [[9,8,7], [0,0,0], 
                         [6,5,4], [3,2,1]]
                       ], dtype = torch.float) # X0 and X1

hx = torch.randn(4, 5) # m X n_neurons
output = []

# for each time step
for i in range(2):
    hx = rnn(X_batch[i], hx)
    output.append(hx)

print(output)

[tensor([[-0.5081,  0.5528, -0.2951,  0.2812, -0.6928],
        [-0.9928,  0.9994, -0.4619, -0.6464, -0.9931],
        [-0.9922,  1.0000,  0.7684, -0.8399, -0.9998],
        [-0.9995,  0.9998,  0.0254, -0.7574, -0.2818]],
       grad_fn=<TanhBackward0>), tensor([[-0.9408,  1.0000,  0.8080,  0.1929, -0.9999],
        [ 0.3481,  0.2554,  0.7809,  0.3738,  0.2357],
        [-0.9017,  0.9998,  0.9392, -0.2323, -0.9909],
        [-0.4607,  0.9650,  0.9392,  0.0098, -0.6245]],
       grad_fn=<TanhBackward0>)]


Now build Pytorch RNN GRAPH

In [ ]:
class MY_RNN(nn.Module):
    def __init__(self, input, neurons):
        super(MY_RNN, self).__init__()
        
        